# STEP 1: Fetch & Structure Tweet Data
### Goal: Get the latest tweets, enrich them with metadata, and append them to the historical record.

API Query 1: Fetch the latest 100 tweets from a specific Twitter list.

API Query 2: For each tweet, fetch public metrics (likes, replies, impressions) and author details.

Convert tweet timestamps into datetime format.

Combine these new tweets with a historical CSV to maintain a growing dataset of all tweets.

Save the full, deduplicated, chronologically sorted dataset to a new CSV file (e.g. 02_04_2025.csv).

# STEP 2: Map Tweets to Game Seasons & Aggregate Stats
### Goal: Attribute each tweet to a game season, and summarize engagement metrics per author.

Define a calendar of game seasons (e.g. Season 4, Season 5, Season 6) using start and end dates.

Convert each tweet’s timestamp and map it to the correct season.

Filter out tweets that fall outside of any season.

Group by season_label and author_username to aggregate:
- tweet_count
- like_count
- impression_count
- reply_count

Save this aggregated view to a CSV file (e.g. season_aggregated_tweets_02_04_2025.csv).

# STEP 3: Publish to Notion
### Goal: Upload the season-by-season summary into a Notion database for easy viewing or sharing.

- Load the aggregated CSV (one row per author per season).
- For each row, create a new page in Notion via the API.

### Each page includes:
- season_label as the title
- author_username as rich text
- All public metrics as number fields
- A placeholder Season Impressions Rank field (can be calculated later)

- Log each entry creation to the console for tracking.

In [2]:
'''
STEP 1 
- API Query 1: Get the 100 most recent tweets by looking at the Twitter list.
- API Query 2: For each of those 100 tweets use the id to query the endpoint to get the public metrics for each of those tweets:.
- Convert the entries in the timestamp column to date-time
- The **newly created dataframe** (the way in which the tweets record is structured) is **combined with the historical one** to get a combined dataframe.
'''

import os
import requests
import json
import pandas as pd
import time
from datetime import datetime
from dotenv import load_dotenv


load_dotenv()

BEARER_TOKEN = os.getenv("BEARER_TOKEN")
HEADERS = {"Authorization": f"Bearer {BEARER_TOKEN}"}


'''
- Fetches the most recent 100 tweets
'''
def fetch_tweets():
    url = "https://api.x.com/2/lists/1651199577985261569/tweets"
    querystring = {"max_results": "100"}

    response = requests.get(url, headers=HEADERS, params=querystring)

    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data.get("data", []))
        return df


'''
- Takes in all the tweets and gets their IDs
- For each tweet 
'''
def get_context_from_post(tweets):
    """Fetch author ID and public metrics from tweet ID"""

    tweet_ids = list(tweets['id'])
    ids_string = ",".join(tweet_ids)

    querystring = {
        "ids": ids_string,
        "tweet.fields": "author_id,public_metrics,created_at"  # Include additional fields as needed
    }

    response = requests.get(
        "https://api.x.com/2/tweets", 
        headers=HEADERS, 
        params=querystring
    )

    context_data = response.json()

    # Creates a new CSV with all the context 
    context_df = pd.DataFrame(context_data['data'])

    df_metrics = context_df['public_metrics'].apply(pd.Series)
    
    context_df.drop(columns=['public_metrics'], inplace=True)

    context_df = context_df.join(df_metrics, how="inner")

    author_ids = []
    for tweet in response.json().get("data", []):
        author_ids.append(tweet.get("author_id"))

    author_ids_string = ",".join(author_ids)

    author_querystring = {
        "ids": author_ids_string,
        "user.fields": "username"
    }
    author_response = requests.get(
        "https://api.x.com/2/users", 
        headers=HEADERS, 
        params=author_querystring
    )

    author_data = author_response.json()
    author_data = author_data['data']

    # Create a DataFrame from the extracted data
    author_df = pd.DataFrame(author_data)

    # Rename the column 'old_name' to 'new_name'
    author_df.rename(columns={'id': 'author_id'}, inplace=True)

    # Joins the author dataframe and the the tweet context dataframe on the author_id
    merged_df = pd.merge(author_df, context_df, left_on='author_id', right_on='author_id', how='inner')

    # Drop irrelevant data 
    merged_df.drop(['name', 'bookmark_count'], axis=1, inplace=True)

    # Updates the name for consistency
    merged_df.rename(columns={'username': 'author_username'}, inplace=True)

    merged_df['edit_history_tweet_ids'] = merged_df['edit_history_tweet_ids'].apply(tuple)
    merged_df.drop_duplicates(inplace=True)

    # Creates a new CSV with the 100 latest tweets and their context 
    merged_df.to_csv("test.csv", index=False)
    
    return merged_df

# Returns a dataframe with three columns: id, edit_history_tweet_ids, text
tweets = fetch_tweets()
# Passes the tweets forward to get their context 
processed_tweets = get_context_from_post(tweets)
# Ensure that the recently updated tweets have the same columns 


'''
This should be maintained so that there is a seperate CSV for each season as well as a CSV containing the global history - for the purposes of pushing to Notion we only want the current season.
'''

# takes ina CSV used as df_bottom
df = pd.read_csv('latest.csv')
processed_tweets = processed_tweets[df.columns]

# Concatenate dataframes with the top one first
df_combined = pd.concat([processed_tweets, df], ignore_index=True)

# Drop duplicates 
df_combined_no_duplicates = df_combined.drop_duplicates()

# Convert the timestamp column to datetime
df_combined_no_duplicates['created_at'] = pd.to_datetime(df_combined_no_duplicates['created_at'], errors='coerce')

df_combined_no_duplicates.sort_values("created_at", ascending=False, inplace=True)

# Save the combined dataframe to a new CSV file
df_combined_no_duplicates.to_csv("02_04_2025.csv", index=False)


In [ ]:
'''
STEP 2
- Define game seasons so that each tweet can be attributed as occuring within a particular game season
- Aggregate the tweet data, to get the tweet counts and public engagement metrics per author per season
- Save the output 
'''

import pandas as pd

csv_file_path = "season6_02_04.csv"  # Replace with actual CSV path
df = pd.read_csv(csv_file_path)

# Check the product calendar to update when there are new seasons.  
game_seasons = [
    {"season_name": "Season 4", "start_date": "2025-01-21", "end_date": "2025-02-19"},
    {"season_name": "Season 5", "start_date": "2025-02-20", "end_date": "2025-03-18"},
    {"season_name": "Season 6", "start_date": "2025-03-25", "end_date": "2025-04-23"},
]

# Convert to DataFrame for easier handling
seasons_df = pd.DataFrame(game_seasons)
seasons_df["start_date"] = pd.to_datetime(seasons_df["start_date"])
seasons_df["end_date"] = pd.to_datetime(seasons_df["end_date"])

# Function to get season for a given date
def get_season(timestamp):
    for _, row in seasons_df.iterrows():
        if row["start_date"] <= timestamp <= row["end_date"]:
            return row["season_name"]
    return "Out of Season"

# Ensure created_at is in datetime format
df["created_at"] = pd.to_datetime(df["created_at"], errors="coerce")
df["created_at"] = df["created_at"].dt.tz_localize(None)

# Check for any rows where created_at could not be parsed
if df["created_at"].isna().sum() > 0:
    print("Warning: Some timestamps could not be parsed. They will be dropped.")

# Drop rows with invalid timestamps
df = df.dropna(subset=["created_at"])

# Map tweets to their season
df["season_label"] = df["created_at"].apply(get_season)

# Drop out-of-season tweets if desired
df = df[df["season_label"] != "Out of Season"]

# Add tweet count
df["tweet_count"] = 1

# Aggregate by season
agg_df = df.groupby(["season_label", "author_username"]).agg(
    tweet_count=("tweet_count", "sum"),
    like_count=("like_count", "sum"),
    impression_count=("impression_count", "sum"),
    reply_count=("reply_count", "sum")
).reset_index()

# Output
agg_df.to_csv("season_aggregated_tweets_02_04_2025.csv", index=False)


In [3]:
'''
STEP 3
- Load the aggregated tweet data from CSV
- For each row (author x season), create a new page in a Notion database
- Each page includes tweet and engagement metrics (impressions, likes, replies) along with metadata like season and author
'''

import pandas as pd 
import requests
import json
from dotenv import load_dotenv
load_dotenv()
NOTION_API_KEY = os.getenv("NOTION_API")

headers = {
    "Authorization": f"Bearer {NOTION_API_KEY}",
    "Notion-Version": "2022-06-28",
    "Content-Type": "application/json",
}

Empty_DB_ID = "1c28ce2d7cac8099ae7ff0c52bb6ed6b"

'''
This CSV needs to contain all the aggregations for the latest season 
'''
csv_file_path = "season_aggregated_tweets_02_04_2025.csv"  
df = pd.read_csv(csv_file_path)


# 🔹 Extract & Prepare Data
for _, row in df.iterrows():
    new_entry = {
        "parent": {"database_id": Empty_DB_ID},
            "properties": {
                "season_label": {
                    "title": [{"text": {"content": row["season_label"]}}]
                },
                "Season Impressions Rank": {"number": 0},

                "author_username": {
                    "rich_text": [{"text": {"content": row["author_username"]}}]
                },
                "tweet_count": {"number": row["tweet_count"]},
                "impression_count": {"number": row["impression_count"]},
                "like_count": {"number": row["like_count"]},
                "reply_count": {"number": row["reply_count"]}
            },
    }

    # Log to the console to see what its being updated to notion 
    print(new_entry) 

    # 🔹 Send request to create a new Notion page
    url = "https://api.notion.com/v1/pages"
    response = requests.post(url, headers=headers, json=new_entry)

    # Print response (check if the row has been successfully added)
    print(f"Added {row['author_username']} - {row['season_label']} → {response.text}")


32
{'parent': {'database_id': '1c28ce2d7cac8099ae7ff0c52bb6ed6b'}, 'properties': {'season_label': {'title': [{'text': {'content': 'Season 6'}}]}, 'Season Impressions Rank': {'number': 0}, 'author_username': {'rich_text': [{'text': {'content': '2026bear'}}]}, 'tweet_count': {'number': 4}, 'impression_count': {'number': 0.0}, 'like_count': {'number': 0.0}, 'reply_count': {'number': 0.0}}}
Added 2026bear - Season 6 → {"object":"page","id":"1c98ce2d-7cac-8103-96c7-d64dfd39aa61","created_time":"2025-04-02T09:01:00.000Z","last_edited_time":"2025-04-02T09:01:00.000Z","created_by":{"object":"user","id":"426970d2-a467-4280-aa05-8f342f45853e"},"last_edited_by":{"object":"user","id":"426970d2-a467-4280-aa05-8f342f45853e"},"cover":null,"icon":null,"parent":{"type":"database_id","database_id":"1c28ce2d-7cac-8099-ae7f-f0c52bb6ed6b"},"archived":false,"in_trash":false,"properties":{"tweet_count":{"id":"%3BRZ%5D","type":"number","number":4},"author_username":{"id":"B%3F%40j","type":"rich_text","rich_te

In [37]:
'''
Aggregates on each week
'''

#Generates a CSV containing the aggregated tweets for each week on record - redundant now that we're looking at seasons rather than weeks 
import pandas as pd

csv_file_path = "latest.csv"  # Replace with actual CSV path
df = pd.read_csv(csv_file_path)

# Ensure created_at is in datetime format
df["created_at"] = pd.to_datetime(df["created_at"], errors="coerce")

# Check for any rows where created_at could not be parsed
if df["created_at"].isna().sum() > 0:
    print("Warning: Some timestamps could not be parsed. They will be dropped.")

# Drop rows with invalid timestamps
df = df.dropna(subset=["created_at"])

# 🔹 Convert 'created_at' to 'week_label' (Monday Start of Week)
df["week_label"] = pd.to_datetime(df["created_at"]).dt.to_period("W").apply(lambda x: x.start_time.date().strftime('%Y-%m-%d'))

df = df[df["author_username"] != "Footium"]

# 🔹 Add a Tweet Count Column (Each row represents one tweet)
df["tweet_count"] = 1

# 🔹 Aggregate Data
agg_df = df.groupby(["week_label", "author_username"]).agg(
    tweet_count=("tweet_count", "sum"),
    like_count=("like_count", "sum"),
    impression_count=("impression_count", "sum"),
    reply_count=("reply_count", "sum")
).reset_index()

agg_df.to_csv("weekly_aggregated_tweets.csv", index=False)


C:\Users\james\AppData\Local\Temp\ipykernel_39988\2141542210.py:18: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df["week_label"] = pd.to_datetime(df["created_at"]).dt.to_period("W").apply(lambda x: x.start_time.date().strftime('%Y-%m-%d'))
